This notebook is a precursor to a script I'll write to tidy the metabolomics data.

In [1]:
import pandas as pd
import numpy as np

#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [30]:
study = 'kuwait-multi-loc'
#'../../../data/proc/metabolomics/'
fmtab = '../../../data/proc/metabolomics/' + study + '.aligned_table.batch_' + study + '.negative.csv'

fclean = '../../../data/clean/mtab.' + study + '.tidy.feather'

In [27]:
df = pd.read_csv(fmtab, index_col=0)
# Dang, there's an extra index column in there
df = df.drop("Unnamed: 0.1", axis=1)

# Rename index to have feature IDs
df.index = ['feat' + str(i) for i in df.index]
df.index.name = 'feat_id'

df.head()

,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,X1,alm_mix_1_ugml__neg,MQ_Blank_neg,...,KW_SSZAHRA_S2_pos,KW_DOWNSTREAM_B2_pos,Alm_Korea_Kuwait_pooled_pos.5,SEOUL_GANGNAM_S3_pos,KW_JABRIYA_BROBOT_pos,SEOUL_SEONGBOK_S3_pos,KW_UPSTREAM_B1_pos,isotopes,adduct,pcgroup
feat_id,,,,,,,,,,,,,,,,,,,,,
feat0,100.573671,100.573660,100.573686,70.7346,67.6954,71.1818,8,8,0.0,0.0,...,0.000000,0.000000,2714.069494,0.000000,0.0,1844.416496,0.0,NaN,NaN,48
feat1,100.634153,100.634142,100.634164,89.6771,89.0626,90.2916,2,2,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,NaN,NaN,6
feat2,101.024448,101.024437,101.024454,89.9803,88.7810,90.2916,7,7,0.0,0.0,...,0.000000,0.000000,0.000000,2102.912058,0.0,0.000000,0.0,NaN,NaN,6
feat3,101.777116,101.777112,101.777136,71.2540,71.0016,74.5088,7,7,0.0,0.0,...,0.000000,0.000000,2206.207607,0.000000,0.0,0.000000,0.0,NaN,NaN,48
feat4,102.033842,102.033827,102.033859,64.5845,63.1256,66.3254,21,21,0.0,0.0,...,174034.510393,1970.056296,117353.662414,132923.872678,0.0,200189.909886,0.0,NaN,NaN,8674


## Convert to tidy

In [28]:
feat_vars = ['feat_id', 'mz', 'mzmin', 'mzmax', 
             'rt', 'rtmin', 'rtmax', 
             'npeaks', 'X1', 
             'isotopes', 'adduct', 'pcgroup']
tidydf = df.reset_index().melt(id_vars=feat_vars, var_name='sample', value_name='intensity')
tidydf.head()

,feat_id,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,X1,isotopes,adduct,pcgroup,sample,intensity
0,feat0,100.573671,100.573660,100.573686,70.7346,67.6954,71.1818,8,8,NaN,NaN,48,alm_mix_1_ugml__neg,0.0
1,feat1,100.634153,100.634142,100.634164,89.6771,89.0626,90.2916,2,2,NaN,NaN,6,alm_mix_1_ugml__neg,0.0
2,feat2,101.024448,101.024437,101.024454,89.9803,88.7810,90.2916,7,7,NaN,NaN,6,alm_mix_1_ugml__neg,0.0
3,feat3,101.777116,101.777112,101.777136,71.2540,71.0016,74.5088,7,7,NaN,NaN,48,alm_mix_1_ugml__neg,0.0
4,feat4,102.033842,102.033827,102.033859,64.5845,63.1256,66.3254,21,21,NaN,NaN,8674,alm_mix_1_ugml__neg,0.0


In [29]:
tidydf.shape, tidydf.dropna(subset=['intensity']).shape

((2621908, 14), (2621908, 14))

## Add sample metadata (from sample IDs)

Parse the sample IDs to add some metadata like country, city, manhole, and sample ID. Also label if things are blanks or pooled samples.

In [40]:
tidydf['sample'].unique()

array(['alm_mix_1_ugml__neg', 'MQ_Blank_neg', 'MQ_Blank_neg.1',
       'MQ_Blank_neg_06.14.17', 'alm_mix_1_ugml__neg.1', 'MQ_Blank_neg.2',
       'Alm_Korea_Kuwait_pooled_condition_neg',
       'Alm_Korea_Kuwait_pooled_condition_neg.1',
       'Alm_Korea_Kuwait_pooled_spike_neg', 'Alm_Korea_Kuwait_pooled_pos',
       'KW_SSZAHRA_Brobot_pos', 'Alm_Korea_Kuwait_pooled_condition_neg.2',
       'Alm_Korea_Kuwait_pooled_neg', 'KW_SSZAHRA_Brobot_neg',
       'KW_SSZAHRA_S3_neg', 'SEOUL_MAPO_S3_neg', 'KW_UPSTREAM_B2_NEG',
       'KW_SSZAHRA_S1_neg', 'KW_SSHITEEN_B2_neg',
       'Alm_Korea_Kuwait_pooled_neg.1', 'KW_JABRIYA_S2_neg',
       'KW_SSHITEEN_S1_neg', 'SEOUL_SEONGBOK_S1_neg',
       'KW_SSHITEEN__B1_neg', 'KW_DOWNSTREAM_S2_neg',
       'SEOUL_SEONGBOK_B2_neg', 'KW_JABRIYA_S1_neg',
       'Alm_Korea_Kuwait_pooled_neg.2', 'SEOUL_GANGNAM_B2_neg',
       'KW_UPSTREAM_S3_NEG', 'KW_JABRIYA_S3_neg', 'KW_SSZAHRA_B1_neg',
       'SEOUL_MAPO_B2_neg', 'KW_DOWNSTREAM_B1_neg',
       'Alm_Korea_Ku

In [43]:
# Start with manhole samples

cities = ['KW', 'SEOUL']

# just an example of the indexing I'm doing here
print(tidydf.loc[ tidydf['sample'].str.contains('|'.join(cities)) ]['sample'].unique())

['KW_SSZAHRA_Brobot_pos' 'KW_SSZAHRA_Brobot_neg' 'KW_SSZAHRA_S3_neg'
 'SEOUL_MAPO_S3_neg' 'KW_UPSTREAM_B2_NEG' 'KW_SSZAHRA_S1_neg'
 'KW_SSHITEEN_B2_neg' 'KW_JABRIYA_S2_neg' 'KW_SSHITEEN_S1_neg'
 'SEOUL_SEONGBOK_S1_neg' 'KW_SSHITEEN__B1_neg' 'KW_DOWNSTREAM_S2_neg'
 'SEOUL_SEONGBOK_B2_neg' 'KW_JABRIYA_S1_neg' 'SEOUL_GANGNAM_B2_neg'
 'KW_UPSTREAM_S3_NEG' 'KW_JABRIYA_S3_neg' 'KW_SSZAHRA_B1_neg'
 'SEOUL_MAPO_B2_neg' 'KW_DOWNSTREAM_B1_neg' 'KW_JABRIYA_B2_neg'
 'SEOUL_GANGNAM_S1_neg' 'KW_DOWNSTREAM_S1_neg' 'KW_SSZAHRA_B2_neg'
 'SEOUL_SEONGBOK_S2_neg' 'KW_SSZAHRA_S2_neg' 'KW_DOWNSTREAM_B2_neg'
 'SEOUL_GANGNAM_S3_neg' 'KW_JABRIYA_BROBOT_neg' 'SEOUL_SEONGBOK_S3_neg'
 'KW_SSHITEEN_S3_neg' 'SEOUL_GANGNAM_B1_neg' 'SEOUL_SEONGBOK_BROBOT_neg'
 'KW_SSHITEEN_S2_neg' 'SEOUL_MAPO_B1_neg' 'SEOUL_MAPO_BROBOT_neg'
 'KW_SSZAHRA_Brobot_pos.1' 'KW_SSZAHRA_S3_pos' 'SEOUL_MAPO_S3_pos'
 'KW_UPSTREAM_B2_pos' 'KW_SSZAHRA_S1_pos' 'KW_SSHITEEN_B2_pos'
 'KW_JABRIYA_S2_pos' 'KW_SSHITEEN_S1_pos' 'SEOUL_SEONGBOK_S1_pos'


In [45]:
manholedf = tidydf.loc[ tidydf['sample'].str.contains('|'.join(cities)) ]

In [49]:
tmp = manholedf.iloc[::100]['sample'].str.split('_', expand=True)
tmp[0].unique(), tmp[1].unique(), tmp[2].unique(), tmp[3].unique(), tmp[4].unique()

(array(['KW', 'SEOUL'], dtype=object),
 array(['SSZAHRA', 'MAPO', 'UPSTREAM', 'SSHITEEN', 'JABRIYA', 'SEONGBOK',
        'DOWNSTREAM', 'GANGNAM'], dtype=object),
 array(['Brobot', 'S3', 'B2', 'S1', 'S2', '', 'B1', 'BROBOT'], dtype=object),
 array(['pos', 'neg', 'NEG', 'B1', 'pos.1'], dtype=object),
 array([None, 'neg', 'pos'], dtype=object))

Ah, oops - have some positive samples in here need to reprocess.